**Zad 1**

In [ ]:
%%sh
cat > lpi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
int main(int argc, char *argv[])
{
  int rank, size;
  MPI_Init( &argc, &argv );
  MPI_Comm_size( MPI_COMM_WORLD, &size );
  MPI_Comm_rank( MPI_COMM_WORLD, &rank );

  long max_n = 1000000;
  double sum = 0.0;
  for (long n = rank; n<max_n; n+=size){
    double tmp = 1.0 / (2.0 * (double)n + 1);
    if (n % 2 == 0){
      sum += tmp;
    } else {
      sum -= tmp;
    }
  }

  double total_sum;
  MPI_Reduce(&sum, &total_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
  if (rank == 0){
    printf("wartosc pi wynosi okolo: %f", 4.0*total_sum);
  }
  MPI_Finalize();
  return 0;
}
EOF
mpicc lpi-mpi.c && mpirun -n 1 --allow-run-as-root a.out

**Zad 2**

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>

double f(double x) {
  return x;
}

int main(int argc, char *argv[]) {

  int np,lp;

  MPI_Status status;
  MPI_Init(&argc, &argv);
  MPI_Comm_rank(MPI_COMM_WORLD, &np);
  MPI_Comm_size(MPI_COMM_WORLD, &lp);


  double xp = 0.0;
  double xk = 1.0;
  int n = 50;

  double clk = 0.0;
  double h = (xk-xp)/n;
  double xi;
  for (long nn = np; nn<n; nn+=lp){
    xi = xp + nn * h;
    clk += f(xi);
  }
  double total;
  MPI_Reduce(&clk, &total, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
  if (np == 0){
    total += f(xp)/2;
    total += f(xk)/2;
    total *= h;
    printf("wartosc calki wynosi ok.: %f", total);
  }
  MPI_Finalize();
  return 0;
}
EOF
mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out

**ZAD 3**

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <stdlib.h>
#include <stdio.h>
#include <mpi.h>
#include <time.h>
#include <sys/time.h>

#define N 39


MPI_Status status;
double A[N][N];
double B[N], C[N];

int main(int argc, char **argv) {
int processCount, processId, slaveTaskCount, source, dest, rows, offset;

struct timeval start, stop;

MPI_Init(&argc, &argv);
MPI_Comm_rank(MPI_COMM_WORLD, &processId);
MPI_Comm_size(MPI_COMM_WORLD, &processCount);

slaveTaskCount = processCount - 1;
if (processId == 0) {

double start = MPI_Wtime();

srand ( time(NULL) );
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    A[i][j]= rand()%10;
  }
  B[i]= rand()%10;
}
printf("\n Mnozenie macierzy przez wektor  \n");
printf("\nMacierz A\n\n");
for (int i = 0; i<N; i++) {
  for (int j = 0; j<N; j++) {
    printf("%.0f\t", A[i][j]);
  }
  printf("\n");
}
printf("\nWektor B\n\n");
for (int i = 0; i<N; i++) {
  printf("%.0f\t", B[i]);
  printf("\n");
}


rows = N/slaveTaskCount;


offset = 0;


for (dest=1; dest <= slaveTaskCount; dest++)
{
  MPI_Send(&offset, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&rows, 1, MPI_INT, dest, 1, MPI_COMM_WORLD);
  MPI_Send(&A[offset][0], rows*N, MPI_DOUBLE,dest,1, MPI_COMM_WORLD);
  MPI_Send(&B, N, MPI_DOUBLE, dest, 1, MPI_COMM_WORLD);
  offset = offset + rows;
}


for (int i = 1; i <= slaveTaskCount; i++)
{
  source = i;
  MPI_Recv(&offset, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&rows, 1, MPI_INT, source, 2, MPI_COMM_WORLD, &status);
  MPI_Recv(&C[offset], rows*N, MPI_DOUBLE, source, 2, MPI_COMM_WORLD, &status);
}

printf("\nWynikowy wektor C = sum(A * B):\n\n");
for (int i = 0; i<N; i++) {
  printf("%.0f\t", C[i]);
  printf ("\n");
}
printf ("\n");

double end = MPI_Wtime();

printf("Czas obliczen %f",end - start);
}

if (processId > 0) {
  source = 0;
  MPI_Recv(&offset, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&rows, 1, MPI_INT, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&A, rows*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  MPI_Recv(&B, N*N, MPI_DOUBLE, source, 1, MPI_COMM_WORLD, &status);
  for (int i = 0; i<rows; i++) {
    for (int j = 0; j<N; j++)
      C[i] = C[i] + A[i][j] * B[j];
  }

  MPI_Send(&offset, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

  MPI_Send(&rows, 1, MPI_INT, 0, 2, MPI_COMM_WORLD);

  MPI_Send(&C, rows*N, MPI_DOUBLE, 0, 2, MPI_COMM_WORLD);
}
MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 40 --allow-run-as-root a.out